<center>
  <h1>Playing with Python Bytecode</h1><br><br>
</center>

In [ ]:
CO_OPTIMIZED          = 0x0001
CO_NEWLOCALS          = 0x0002
CO_VARARGS            = 0x0004
CO_VARKEYWORDS        = 0x0008
CO_NESTED             = 0x0010
CO_GENERATOR          = 0x0020
CO_NOFREE             = 0x0040
CO_COROUTINE          = 0x0080
CO_ITERABLE_COROUTINE = 0x0100

In [ ]:
CO_FUTURE_DIVISION         = 0x2000
CO_FUTURE_ABSOLUTE_IMPORT  = 0x4000
CO_FUTURE_WITH_STATEMENT   = 0x8000
CO_FUTURE_PRINT_FUNCTION   = 0x10000
CO_FUTURE_UNICODE_LITERALS = 0x20000
CO_FUTURE_BARRY_AS_BDFL    = 0x40000
CO_FUTURE_GENERATOR_STOP   = 0x80000

In [ ]:
# def addone(x):
#     return x + 1

my_code = CodeType(1,             # argcount
                   0,             # kwonlyargcount
                   1,             # nlocals
                   2,             # stacksize
                   (CO_OPTIMIZED | CO_NEWLOCALS | CO_NOFREE),
# Resume Typing
                   